# OST Writing Assessment: Feature Selection Analysis

VIF-guided elastic net feature selection for predicting writing quality.

**Reproducibility**: Results may vary slightly (±0.01 R²) due to CV fold assignments. Use `random_state=42`.

**Key Parameters**:
- VIF threshold: 10
- Bootstrap: 1,000 iterations, selection threshold |β| > 0.05
- Elastic Net: L1 ratios [0.1, 0.3, 0.5, 0.7, 0.9], 10-fold CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [ ]:
df = pd.read_csv('Writing_Assessment_Cleaned_Dataset_Tool_Specify.csv')
target = 'Writing_Quality_Sum_Score'
ost_features = [col for col in df.columns if col.startswith('OST_')]

print(f"N = {len(df)}, OST features = {len(ost_features)}")
print(f"Target: M = {df[target].mean():.2f}, SD = {df[target].std():.2f}")
print(f"Native (0): n={sum(df['Primary_Language']==0)}, Non-native (1): n={sum(df['Primary_Language']==1)}")

## VIF Functions

In [ ]:
def calculate_vif(X):
    X_scaled = StandardScaler().fit_transform(X)
    vif = []
    for i in range(X_scaled.shape[1]):
        try:
            v = variance_inflation_factor(X_scaled, i)
            v = v if np.isfinite(v) and v < 1e10 else np.inf
        except:
            v = np.inf
        vif.append({'feature': X.columns[i], 'VIF': v})
    return pd.DataFrame(vif).sort_values('VIF', ascending=False)

def iterative_vif_removal(df, features, threshold=10):
    current = features.copy()
    removed = []
    while True:
        X = df[current].dropna()
        if X.shape[1] < 2:
            break
        vif_df = calculate_vif(X)
        if vif_df.iloc[0]['VIF'] <= threshold:
            break
        feat = vif_df.iloc[0]['feature']
        removed.append((feat, vif_df.iloc[0]['VIF']))
        current.remove(feat)
        print(f"  Removed: {feat} (VIF={vif_df.iloc[0]['VIF']:.1f})")
    return current, removed

## Feature Selection

In [ ]:
y = df[target]

# Top 15 by |correlation|
correlations = [(f, abs(df[f].corr(y)), df[f].corr(y)) for f in ost_features]
correlations.sort(key=lambda x: x[1], reverse=True)

print("Top 15 features by |r|:")
for i, (f, _, r) in enumerate(correlations[:15], 1):
    print(f"  {i:2d}. {f:40s} r={r:+.3f}")

In [ ]:
top_15 = [f for f, _, _ in correlations[:15]]
final_features, removed = iterative_vif_removal(df, top_15.copy(), threshold=10)
print(f"\nFinal: {len(final_features)} features")

## Elastic Net Model

In [ ]:
X = df[final_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = ElasticNetCV(
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
    alphas=np.logspace(-3, 1, 30),
    cv=10, max_iter=5000, random_state=42
)
model.fit(X_scaled, y)

y_pred = model.predict(X_scaled)
r2 = r2_score(y, y_pred)
n, p = len(y), len(final_features)
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
mse = mean_squared_error(y, y_pred)
aic = n * np.log(mse) + 2 * p
bic = n * np.log(mse) + p * np.log(n)

print(f"N={n}, Features={p}")
print(f"R²={r2:.3f}, Adj R²={adj_r2:.3f}")
print(f"AIC={aic:.1f}, BIC={bic:.1f}")

In [ ]:
# VIF check
vif_final = calculate_vif(X)
print(f"VIF: mean={vif_final['VIF'].mean():.2f}, max={vif_final['VIF'].max():.2f}")

# Coefficients
print("\nCoefficients:")
coef_df = pd.DataFrame({'feature': final_features, 'coef': model.coef_})
coef_df = coef_df.sort_values('coef', key=abs, ascending=False)
for _, row in coef_df.iterrows():
    print(f"  {row['feature']:45s} β={row['coef']:+.3f}")

## Bootstrap Validation

In [ ]:
# Selection threshold |β| > 0.05 excludes coefficients with negligible practical significance
THRESH = 0.05
n_boot = 1000

boot_r2 = []
selection_count = {f: 0 for f in final_features}

for i in range(n_boot):
    idx = np.random.choice(len(y), len(y), replace=True)
    m = ElasticNet(alpha=model.alpha_, l1_ratio=model.l1_ratio_, max_iter=5000)
    m.fit(X_scaled[idx], y.iloc[idx])
    boot_r2.append(r2_score(y.iloc[idx], m.predict(X_scaled[idx])))
    for j, f in enumerate(final_features):
        if abs(m.coef_[j]) > THRESH:
            selection_count[f] += 1

boot_r2 = np.array(boot_r2)
print(f"Bootstrap R² 95% CI: [{np.percentile(boot_r2, 2.5):.3f}, {np.percentile(boot_r2, 97.5):.3f}]")

cv_scores = cross_val_score(model, X_scaled, y, cv=10, scoring='r2')
print(f"CV R² = {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

In [ ]:
print(f"Feature stability (|β|>{THRESH}):")
stability = pd.DataFrame([{'feature': f, 'freq': c/n_boot} for f, c in selection_count.items()])
stability = stability.sort_values('freq', ascending=False)
for _, row in stability.iterrows():
    mark = "*" if row['freq'] >= 0.80 else ""
    print(f"  {row['feature']:45s} {row['freq']*100:5.1f}% {mark}")
print(f"\nFeatures ≥80%: {sum(stability['freq']>=0.80)}")

## Group-Specific Analysis

In [ ]:
native = df['Primary_Language'] == 0
nonnative = df['Primary_Language'] == 1

t, p_val = stats.ttest_ind(df.loc[native, target], df.loc[nonnative, target])
print(f"Native: M={df.loc[native, target].mean():.2f}, SD={df.loc[native, target].std():.2f}")
print(f"Non-native: M={df.loc[nonnative, target].mean():.2f}, SD={df.loc[nonnative, target].std():.2f}")
print(f"t({len(df)-2})={t:.3f}, p={p_val:.3f}")

In [ ]:
# Native model (top 6 features)
native_df = df[native]
y_n = native_df[target]
corrs_n = [(f, abs(native_df[f].corr(y_n))) for f in ost_features]
corrs_n.sort(key=lambda x: x[1], reverse=True)
native_feats = [f for f, _ in corrs_n[:6]]

X_n = StandardScaler().fit_transform(native_df[native_feats])
m_n = ElasticNetCV(l1_ratio=[0.1,0.3,0.5,0.7,0.9], alphas=np.logspace(-2,1,20), cv=5, max_iter=5000, random_state=42)
m_n.fit(X_n, y_n)

r2_n = r2_score(y_n, m_n.predict(X_n))
adj_r2_n = 1 - (1 - r2_n) * (len(y_n) - 1) / (len(y_n) - 6 - 1)

print(f"Native: n={len(y_n)}, k=6, R²={r2_n:.3f}, Adj R²={adj_r2_n:.3f}")
for f, c in sorted(zip(native_feats, m_n.coef_), key=lambda x: abs(x[1]), reverse=True):
    print(f"  {f:40s} β={c:+.3f}")

In [ ]:
# Non-native model (3 features)
nn_feats = ['OST_flesch_kincaid_grade_level', 'OST_context_sensitive_count', 'OST_error_count']
nn_df = df[nonnative]
y_nn = nn_df[target]

X_nn = StandardScaler().fit_transform(nn_df[nn_feats])
m_nn = ElasticNetCV(l1_ratio=[0.1,0.3,0.5,0.7,0.9], alphas=np.logspace(-2,1,20), cv=5, max_iter=5000, random_state=42)
m_nn.fit(X_nn, y_nn)

r2_nn = r2_score(y_nn, m_nn.predict(X_nn))
adj_r2_nn = 1 - (1 - r2_nn) * (len(y_nn) - 1) / (len(y_nn) - 3 - 1)

print(f"Non-native: n={len(y_nn)}, k=3, R²={r2_nn:.3f}, Adj R²={adj_r2_nn:.3f}")
for f, c in sorted(zip(nn_feats, m_nn.coef_), key=lambda x: abs(x[1]), reverse=True):
    print(f"  {f:40s} β={c:+.3f}")

## Figures

In [ ]:
# Figure 1: Feature Importance and Bootstrap Stability
coef_plot = coef_df.sort_values('coef', key=abs, ascending=True)
stab_plot = stability.sort_values('freq', ascending=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

ax1 = axes[0]
colors1 = ['#2ecc71' if c > 0 else '#e74c3c' for c in coef_plot['coef']]
ax1.barh(range(len(coef_plot)), coef_plot['coef'], color=colors1, alpha=0.85)
ax1.set_yticks(range(len(coef_plot)))
ax1.set_yticklabels([f.replace('OST_', '').replace('_', ' ') for f in coef_plot['feature']], fontsize=10)
ax1.axvline(0, color='black', lw=0.8)
ax1.set_xlabel('Standardized Coefficient (β)')
ax1.set_title('(A) Feature Importance', fontweight='bold')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax2 = axes[1]
ax2.barh(range(len(stab_plot)), stab_plot['freq'], color='#3498db', alpha=0.85)
ax2.set_yticks(range(len(stab_plot)))
ax2.set_yticklabels([f.replace('OST_', '').replace('_', ' ') for f in stab_plot['feature']], fontsize=10)
ax2.axvline(0.8, color='#e74c3c', ls='--', lw=2, label='80% threshold')
ax2.set_xlabel('Selection Frequency')
ax2.set_title('(B) Bootstrap Stability (1,000 iterations)', fontweight='bold')
ax2.set_xlim(0, 1.05)
ax2.legend(loc='lower right')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
for i, (_, row) in enumerate(stab_plot.iterrows()):
    ax2.text(row['freq']+0.02, i, f"{row['freq']*100:.0f}%", va='center', fontsize=9)

plt.tight_layout()
plt.savefig('Figure1_Feature_Importance_Stability.png', dpi=300, bbox_inches='tight')
plt.savefig('Figure1_Feature_Importance_Stability.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Figure 2: Group-Specific Feature Coefficients
native_coefs = pd.DataFrame({'feature': native_feats, 'coef': m_n.coef_}).sort_values('coef', key=abs, ascending=True)
nn_coefs = pd.DataFrame({'feature': nn_feats, 'coef': m_nn.coef_}).sort_values('coef', key=abs, ascending=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
c1 = ['#4CAF50' if c > 0 else '#F44336' for c in native_coefs['coef']]
ax1.barh(range(len(native_coefs)), native_coefs['coef'], color=c1, height=0.65)
ax1.set_yticks(range(len(native_coefs)))
ax1.set_yticklabels([f.replace('OST_', '').replace('_', '_') for f in native_coefs['feature']], fontsize=10)
ax1.axvline(0, color='black', lw=0.8)
ax1.set_xlabel('Standardized Coefficient')
ax1.set_title(f'Native Speakers (n={len(y_n)}, 6 features)\nR²={r2_n:.3f}, Adj R²={adj_r2_n:.3f}')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax2 = axes[1]
c2 = ['#4CAF50' if c > 0 else '#F44336' for c in nn_coefs['coef']]
ax2.barh(range(len(nn_coefs)), nn_coefs['coef'], color=c2, height=0.65)
ax2.set_yticks(range(len(nn_coefs)))
ax2.set_yticklabels([f.replace('OST_', '').replace('_', '_') for f in nn_coefs['feature']], fontsize=10)
ax2.axvline(0, color='black', lw=0.8)
ax2.set_xlabel('Standardized Coefficient')
ax2.set_title(f'Non-Native Speakers (n={len(y_nn)}, 3 features)\nR²={r2_nn:.3f}, Adj R²={adj_r2_nn:.3f}')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('Figure2_Group_Coefficients.png', dpi=300, bbox_inches='tight')
plt.savefig('Figure2_Group_Coefficients.pdf', dpi=300, bbox_inches='tight')
plt.show()

## Summary

In [ ]:
print(f"""
FULL MODEL (N={n}):
  Features: {p}
  R²={r2:.3f}, Adj R²={adj_r2:.3f}
  Bootstrap R² 95% CI: [{np.percentile(boot_r2, 2.5):.3f}, {np.percentile(boot_r2, 97.5):.3f}]
  CV R²={cv_scores.mean():.3f}±{cv_scores.std():.3f}
  Stable features (≥80%): {sum(stability['freq']>=0.80)}

GROUP-SPECIFIC:
  Native (n={len(y_n)}): 6 features, R²={r2_n:.3f}, Adj R²={adj_r2_n:.3f}
  Non-native (n={len(y_nn)}): 3 features, R²={r2_nn:.3f}, Adj R²={adj_r2_nn:.3f}
""")